In [105]:
import wrds
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import preprocessing, tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
pd.set_option('display.max_columns', None)
import statsmodels.api as sm
from statsmodels.api import add_constant
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings("ignore")

In [106]:
db = wrds.Connection(wrds_username = 'rotmanren') # Change to user's username

Loading library list...
Done


In [107]:
# Load all data
startYear = 2009 #!!!!!!!!!!2009 or 2010, maybe 2010?
# Accruals need data from 2009 to obtain changes
endYear = 2022
dataquery1 = f"""
            SELECT a.DATADATE, a.cik, a.GVKEY, a.tic, a.fyear, a.ib, a.spi, a.at, a.dvc, a.act, a.che, a.lct, a.dlc, a.txp, a.dp, a.csho, a.ceq, a.ivao, a.lt, a.ajex, a.dltt, a.ivst, a.pstk, b.cshom, b.prccm
            FROM comp.funda as a
            JOIN comp_na_daily_all.secm as b
            ON a.tic = b.tic AND a.fyear = b.cyear
            WHERE consol = 'C'
            AND popsrc = 'D'
            AND indfmt = 'INDL'
            AND datafmt = 'STD' 
            AND fyear BETWEEN '{startYear}' AND '{endYear}'
            AND b.cmth = 6.0
        """
inforaw = db.raw_sql(dataquery1)
# Fill NA for selected variables 
cols = ['spi', 'dvc', 'che', 'lct', 'dlc', 'txp', 'dp', 'ceq', 'ivao', 'lt', 'dltt', 'ivst', 'pstk']
inforaw[cols] = inforaw[cols].fillna(0)
inforaw = inforaw.dropna()
inforaw = inforaw.reset_index()
# Adjust for stock split
inforaw['csho'] = inforaw['csho'] * inforaw['ajex']
inforaw['mrk_eq'] = inforaw['cshom'] * inforaw['prccm']
inforaw


,index,datadate,cik,gvkey,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,txp,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq
0,0,2010-05-31,0000001750,001004,AIR,2009.0,44.628,-4.302,1501.042,0.000,863.429,79.370,325.550,100.833,3.263,38.930,39.484,746.906,2.143,754.692,1.0,336.191,0.0,0.0,38697000.0,16.050,6.210868e+08
1,1,2011-05-31,0000001750,001004,AIR,2010.0,73.139,-1.536,1703.727,2.983,913.985,57.433,416.010,114.075,0.000,59.296,39.781,835.845,2.443,868.438,1.0,329.802,0.0,0.0,39025000.0,16.740,6.532785e+08
2,2,2012-05-31,0000001750,001004,AIR,2011.0,67.723,-13.864,2195.653,12.081,1063.272,67.720,473.226,122.865,0.000,80.333,40.273,864.649,18.869,1329.631,1.0,669.489,0.0,0.0,39739000.0,27.090,1.076530e+09
3,3,2013-05-31,0000001750,001004,AIR,2012.0,55.000,-21.100,2136.900,11.900,1033.700,75.300,389.000,86.400,0.000,108.600,39.382,918.600,16.800,1217.400,1.0,622.200,0.0,0.0,40288000.0,13.480,5.430822e+08
4,4,2014-05-31,0000001750,001004,AIR,2013.0,72.900,0.000,2199.500,11.800,1116.900,89.200,402.100,69.700,0.000,113.400,39.560,999.500,5.200,1198.800,1.0,564.300,0.0,0.0,39733000.0,21.980,8.733313e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68862,127145,2019-12-31,0001280776,331856,IMUX,2019.0,-34.933,-9.966,65.955,0.000,32.230,29.369,7.072,0.000,0.000,0.050,10.745,58.363,0.000,7.592,1.0,0.520,0.0,0.0,9986000.0,11.470,1.145394e+08
68863,127146,2022-12-31,0001857190,339950,NYXH,2022.0,-33.371,0.000,156.112,0.000,107.152,19.118,15.938,1.183,3.905,2.066,25.846,128.595,0.185,27.517,1.0,11.516,0.0,0.0,25797000.0,9.200,2.373324e+08
68864,127147,2021-12-31,0001566373,345556,FSTX,2021.0,-31.283,-1.337,123.021,0.000,84.739,78.549,12.135,0.906,0.000,0.642,20.875,96.857,0.000,26.164,1.0,12.328,0.0,0.0,19366000.0,8.590,1.663539e+08
68865,127148,2014-06-30,0001345099,271607,MESO,2014.0,-75.534,-2.859,847.153,0.000,191.931,185.003,40.199,0.000,0.000,1.024,63.431,538.559,0.000,308.594,1.0,0.000,0.0,0.0,321640000.0,21.095,6.784996e+09


In [108]:
# Ignore
# dataquery1 = f"""
#             SELECT *
#             FROM comp_na_daily_all.secm
#             LIMIT 10
#         """
# db.raw_sql(dataquery1)


In [109]:
# # Load analyst data for three years
# startYear_an = 2019
# endYear_an = 2019
# dataquery_one = f"""
#             SELECT  (a.ticker), (EXTRACT(YEAR FROM a.fpedats)) as predict_year, a. fpedats, a. statpers, a.actual, a.medest
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '1'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# analystraw = db.raw_sql(dataquery_one)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# df_analyst_one = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_analyst_one = df_analyst_one.dropna()
# df_analyst_one = df_analyst_one.reset_index()

# # print(df_analyst_one.head(20))

# dataquery_two = f"""
#             SELECT  (a.ticker), (EXTRACT(YEAR FROM a.fpedats)) as predict_year, a. fpedats, a. statpers, a.actual, a.medest
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '2'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# analystraw = db.raw_sql(dataquery_two)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# df_analyst_two = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_analyst_two = df_analyst_two.dropna()
# df_analyst_two = df_analyst_two.reset_index()

# # print(df_analyst_two.head(20))

# dataquery_three = f"""
#             SELECT  (a.ticker), (EXTRACT(YEAR FROM a.fpedats)) as predict_year, a. fpedats, a. statpers, a.actual, a.medest
#             FROM ibes.statsum_epsus as a
#             WHERE 
#             fpi = '3'
#             AND measure = 'EPS'
#             AND EXTRACT(YEAR FROM a.statpers) BETWEEN '{startYear_an}' AND '{endYear_an}'
#             AND EXTRACT(MONTH FROM a.statpers) = 6
#         """
# analystraw = db.raw_sql(dataquery_three)
# linkdf = pd.read_excel("IBES_Compustat_Link_File.xls")
# df_analyst_three = pd.merge(analystraw, linkdf, how='inner', left_on=['ticker'], right_on=['TICKER'])
# df_analyst_three = df_analyst_three.dropna()
# df_analyst_three = df_analyst_three.reset_index()

# # print(df_analyst_three.head(20))

In [110]:
# df_analyst_one = pd.merge(inforaw, df_analyst_two, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
# df_analyst_one['Earning'] = df_analyst_one['ib'] - df_analyst_one['spi']
# df_analyst_one['flat_actual'] = df_analyst_one['actual'] * df_analyst_one['csho']
# df_analyst_one

In [111]:
# df_analyst_one = pd.merge(inforaw, df_analyst_three, how='inner', left_on=['tic', 'datadate'], right_on=['ticker', 'fpedats'])
# df_analyst_one.head(30)

,index_x,datadate,cik,gvkey_x,tic,fyear,ib,spi,at,dvc,act,che,lct,dlc,txp,dp,csho,ceq,ivao,lt,ajex,dltt,ivst,pstk,cshom,prccm,mrk_eq,index_y,ticker,predict_year,fpedats,statpers,actual,medest,gvkey_y,TICKER
0,11,2021-05-31,0000001750,001004,AIR,2020.0,46.300,16.100,1539.700,0.100,937.000,60.200,336.800,11.500,0.700,36.300,35.375,974.400,29.900,565.300,1.0,193.600,8.400,0.0,3.510100e+07,20.67,7.255377e+08,1208,AIR,2021.0,2021-05-31,2019-06-20,1.310,3.06,1004,AIR
1,113,2021-12-31,0000001800,001078,ABT,2021.0,7071.000,-539.000,75196.000,3235.000,24239.000,10249.000,13105.000,999.000,306.000,3538.000,1764.082,35802.000,169.000,39172.000,1.0,18252.000,450.000,0.0,1.776820e+09,115.93,2.059867e+11,266,ABT,2021.0,2021-12-31,2019-06-20,5.210,4.03,1078,ABT
2,317,2021-09-30,0000002969,001209,APD,2021.0,2028.800,13.600,26859.200,1292.600,8376.300,5800.800,2799.100,565.500,93.900,1321.300,221.397,13539.700,807.200,12771.200,1.0,7653.700,1331.900,0.0,2.213140e+08,287.68,6.366761e+10,291,APD,2021.0,2021-09-30,2019-06-20,9.020,10.46,1209,APD
3,404,2021-12-31,0000002488,001161,AMD,2021.0,3162.000,-57.000,12419.000,0.000,8583.000,3608.000,4240.000,383.000,0.000,296.000,1207.000,7497.000,0.000,4922.000,1.0,349.000,1073.000,0.0,1.215021e+09,93.93,1.141269e+11,518,AMD,2021.0,2021-12-31,2019-06-20,2.790,1.03,1161,AMD
4,463,2021-10-31,0000771497,001410,ABM,2021.0,126.300,-200.100,4436.200,51.000,1401.100,62.800,1285.800,63.600,11.400,89.900,67.302,1609.200,0.000,2827.000,1.0,971.400,0.000,0.0,6.713100e+07,44.35,2.977260e+09,727,ABM,2021.0,2021-10-31,2019-06-20,3.580,2.52,1410,ABM
5,486,2021-12-31,0000766421,001230,ALK,2021.0,478.000,925.000,13951.000,0.000,3920.000,3116.000,3991.000,634.000,0.000,394.000,125.906,3801.000,0.000,10150.000,1.0,3452.000,2646.000,0.0,1.244820e+08,60.31,7.507509e+09,989,ALK,2021.0,2021-12-31,2019-06-20,-2.030,7.56,1230,ALK
6,562,2021-12-31,0001783400,001439,ECOL,2021.0,5.337,-3.556,1805.398,0.000,364.218,67.487,186.934,24.127,0.559,110.776,31.270,624.324,11.400,1181.074,1.0,775.710,0.000,0.0,3.151200e+07,37.52,1.182330e+09,607,ECOL,2021.0,2021-12-31,2019-06-20,0.170,3.21,1439,ECOL
7,575,2021-12-31,0000004904,001440,AEP,2021.0,2488.100,39.000,87668.700,1507.700,7809.200,671.800,12426.700,5170.000,1586.400,2825.700,504.212,22433.200,3867.000,64945.200,1.0,31989.600,268.400,43.3,4.997500e+08,84.59,4.227385e+10,1420,AEP,2021.0,2021-12-31,2019-06-20,4.740,4.68,1440,AEP
8,726,2021-09-30,0000320193,001690,AAPL,2021.0,94680.000,0.000,351002.000,14431.000,134836.000,62639.000,125481.000,17141.000,0.000,9500.000,16426.786,63090.000,127877.000,287912.000,1.0,119381.000,27699.000,0.0,1.668763e+10,136.96,2.285538e+12,1201,AAPL,2021.0,2021-09-30,2019-06-20,5.610,3.64,1690,AAPL
9,740,2021-10-31,0000006951,001704,AMAT,2021.0,5888.000,-380.000,25825.000,851.000,16107.000,5459.000,6344.000,73.000,734.000,394.000,892.000,12247.000,2055.000,13578.000,1.0,5680.000,464.000,0.0,9.176610e+08,142.40,1.306749e+11,284,AMAT,2021.0,2021-10-31,2019-06-20,6.840,4.17,1704,AMAT


In [112]:
df_analyst_three.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2454 entries, 0 to 2453
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   index         2454 non-null   int64  
 1   ticker        2454 non-null   object 
 2   predict_year  2454 non-null   float64
 3   fpedats       2454 non-null   object 
 4   statpers      2454 non-null   object 
 5   actual        2454 non-null   float64
 6   medest        2454 non-null   float64
 7   gvkey         2454 non-null   int64  
 8   TICKER        2454 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 172.7+ KB


## ADV preparation

In [113]:
# A function taking care of all financial values calculation
def  calValues(db, company_list):
    print('Estimated waiting time 4mins, please wait')
    # Prepare dataset for nine models
    # HVZ one year
    df_HVZ_one = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_1'])
    # HVZ two year
    df_HVZ_two = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_2'])
    # HVZ three year
    df_HVZ_three = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC', 'E_3'])

    # EP one year
    df_EP_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_1'])
    # EP two year
    df_EP_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_2'])
    # EP three year
    df_EP_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'E_3'])

    # RI one year
    df_RI_one = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_1'])
    # RI two year
    df_RI_two = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_2'])
    # RI three year
    df_RI_three = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'E_3'])

    X_pred_HVZs = pd.DataFrame(columns=['at', 'dvc', 'DD', 'E', 'NegE', 'AC'])
    X_pred_EPs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E'])
    X_pred_RIs = pd.DataFrame(columns=['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC'])

    Y_pred_one = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_two = pd.DataFrame(columns=['tic','E_deflat'])
    Y_pred_three = pd.DataFrame(columns=['tic','E_deflat'])
    count = 0
    for cik in company_list['cik'].astype(str):
        # print(count)
        count += 1
        # accrualsinfo = db.raw_sql(dataquery)
        infocomp = inforaw.loc[(inforaw['cik'] == cik)].reset_index(drop=True)
        infocomp = infocomp.sort_values(by='fyear')
    

        # HVZ model
        infocomp['E'] = infocomp['ib'] - infocomp['spi']
        infocomp['DD'] = (infocomp['dvc'] > 0).astype(int)
        infocomp['NegE'] = (infocomp['E'] < 0).astype(int)
        infocomp['AC'] = (infocomp['act'] - infocomp['che']).diff() - (infocomp['lct'] - infocomp['dlc'] - infocomp['txp']).diff() - infocomp['dp']
        # The HVZ needs to be devided by CSHO prior to be compared with the rest

        # EP model
        infocomp['E_deflat'] = infocomp['E'] / infocomp['csho']
        infocomp['NegE_E'] = infocomp['E_deflat'] * infocomp['NegE']

        # RI model
        infocomp['B'] = infocomp['ceq'] / infocomp['csho']
        infocomp['WC'] = (infocomp['act'] - infocomp['che']) - (infocomp['lct'] - infocomp['dlc'])
        infocomp['NCO'] = (infocomp['at'] - infocomp['act'] - infocomp['ivao']) - (infocomp['lt'] - infocomp['lct'] - infocomp['dltt'])
        infocomp['FIN'] = ((infocomp['ivst'] - infocomp['ivao']) - (infocomp['dltt'] - infocomp['dlc'] - infocomp['pstk'])) 
        infocomp['TACC'] = (infocomp['WC'] + infocomp['NCO'] + infocomp['FIN']) / infocomp['csho']

        # Compile dataset
        # Drop year differnece axillary 
        infocomp = infocomp[infocomp['fyear'] != 2009]
        # Year range of training
        infocomptrain = infocomp[(infocomp['fyear'] >= 2010) & (infocomp['fyear'] <= 2019)].reset_index(drop=True)
        # Prediction data
        X_pred_2019 = infocomptrain.loc[infocomptrain['fyear'] == 2019].reset_index(drop=True)
        X_pred_2019 = X_pred_2019.dropna()
        if X_pred_2019.empty is not None: 
            # Yearly data
            Y_pred_2020 = infocomp[(infocomp['fyear'] == 2020)].reset_index(drop=True)
            Y_pred_2020 = Y_pred_2020[['tic','E_deflat']]
            Y_pred_one = Y_pred_one.append(Y_pred_2020, ignore_index=True)

            Y_pred_2021 = infocomp[(infocomp['fyear'] == 2021)].reset_index(drop=True)
            Y_pred_2021 = Y_pred_2021[['tic','E_deflat']]
            Y_pred_two = Y_pred_two.append(Y_pred_2021, ignore_index=True)

            Y_pred_2022 = infocomp[(infocomp['fyear'] == 2022)].reset_index(drop=True)
            Y_pred_2022 = Y_pred_2022[['tic','E_deflat']]
            Y_pred_three = Y_pred_three.append(Y_pred_2022, ignore_index=True)
        ## Calculate HVZ Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E']].reset_index(drop=True)
        
        X_pred_HVZ = X_pred_2019[['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_HVZs = X_pred_HVZs.append(X_pred_HVZ, ignore_index=True)

        y = y.rename(columns={"E": "E_1"})
        HVZ_one = pd.concat([X, y], axis=1)
        df_HVZ_one = df_HVZ_one.append(HVZ_one, ignore_index=True)
        df_HVZ_one = df_HVZ_one.dropna()

        ## Calculate HVZ Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_2"})
        HVZ_two = pd.concat([X, y], axis=1)
        df_HVZ_two = df_HVZ_two.append(HVZ_two, ignore_index=True)
        df_HVZ_two = df_HVZ_two.dropna()

        ## Calculate HVZ Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['at', 'dvc', 'DD', 'E','NegE', 'AC', 'csho']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E']].reset_index(drop=True)
        y = y.rename(columns={"E": "E_3"})
        HVZ_three = pd.concat([X, y], axis=1)
        df_HVZ_three = df_HVZ_three.append(HVZ_three, ignore_index=True)
        df_HVZ_three = df_HVZ_three.dropna()

        ## Calculate EP Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)

        X_pred_EP = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_EPs = X_pred_EPs.append(X_pred_EP, ignore_index=True)

        y = y.rename(columns={"E_deflat": "E_1"})
        EP_one = pd.concat([X, y], axis=1)
        df_EP_one = df_EP_one.append(EP_one, ignore_index=True)
        df_EP_one = df_EP_one.dropna()

        ## Calculate EP Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        EP_two = pd.concat([X, y], axis=1)
        df_EP_two = df_EP_two.append(EP_two, ignore_index=True)
        df_EP_two = df_EP_two.dropna()

        ## Calculate EP Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        EP_three = pd.concat([X, y], axis=1)
        df_EP_three = df_EP_three.append(EP_three, ignore_index=True)
        df_EP_three = df_EP_three.dropna()

        ## Calculate RI Prediction, One year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2018, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2011, ['E_deflat']].reset_index(drop=True)
        
        X_pred_RI = X_pred_2019[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC', 'gvkey', 'tic']].reset_index(drop=True)
        X_pred_RIs = X_pred_RIs.append(X_pred_RI, ignore_index=True)
        
        y = y.rename(columns={"E_deflat": "E_1"})
        RI_one = pd.concat([X, y], axis=1)
        df_RI_one = df_RI_one.append(RI_one, ignore_index=True)
        df_RI_one = df_RI_one.dropna()

        ## Calculate RI Prediction, Two year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2017, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2012, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_2"})
        RI_two = pd.concat([X, y], axis=1)
        df_RI_two = df_RI_two.append(RI_two, ignore_index=True)
        df_RI_two = df_RI_two.dropna()

        ## Calculate RI Prediction, Three year gap
        X = infocomptrain.loc[infocomptrain['fyear'] <= 2016, ['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']].reset_index(drop=True)
        y = infocomptrain.loc[infocomptrain['fyear'] >= 2013, ['E_deflat']].reset_index(drop=True)
        y = y.rename(columns={"E_deflat": "E_3"})
        RI_three = pd.concat([X, y], axis=1)
        df_RI_three = df_RI_three.append(RI_three, ignore_index=True)
        df_RI_three = df_RI_three.dropna()
    return df_HVZ_one, df_HVZ_two, df_HVZ_three, df_EP_one, df_EP_two, df_EP_three, df_RI_one, df_RI_two, df_RI_three, X_pred_HVZs, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three
        # infocomptrain


In [114]:
#Compile companylist for companies that EXISTED in EVERY year between startyeartrain to endyeartest to not Nan the value
year = startYear
query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
company_list = db.raw_sql(query_comp) 
for year in np.arange(startYear,endYear):
    query_comp = (
        f"""
        SELECT DISTINCT cik
        FROM comp.funda
        WHERE fyear = {year}
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        """) 
    comp_list = db.raw_sql(query_comp) 
    company_list = pd.merge(comp_list,company_list, on= 'cik', how='inner')
company_list = company_list.dropna().reset_index(drop=True)

# Company list obtained, Get company tic for reviewing convenience
query_comp = (
        f"""
        SELECT DISTINCT cik, MIN (tic) AS tic
        FROM comp.funda
        WHERE fyear BETWEEN '{startYear}' AND '{endYear}'
        AND consol = 'C'
        AND popsrc = 'D'
        AND indfmt = 'INDL'
        AND datafmt = 'STD'  
        group by cik
        """) 
company_tic_list = db.raw_sql(query_comp) 
company_list = pd.merge(company_list, company_tic_list, left_on = 'cik', right_on = 'cik', how='left')
# Make sure we have the tic exist in linkdf so we can find analyst data
company_list = pd.merge(company_list, linkdf, left_on = 'tic', right_on = 'TICKER', how='inner')

print(company_list)

             cik   tic   gvkey TICKER
0     0001119774  CGEN    3172   CGEN
1     0001115837   MBT  137433    MBT
2     0000075362  PCAR    8253   PCAR
3     0001395942   KAR  183581    KAR
4     0000871763   MAN    7007    MAN
...          ...   ...     ...    ...
1854  0001289848  HURN  161997   HURN
1855  0001084048    ZD   66716     ZD
1856  0001345105   CPA   21381    CPA
1857  0000882361  APTO   15490   APTO
1858  0001046025  HFWA   66285   HFWA

[1859 rows x 4 columns]


In [115]:
# Data extraction
df_HVZ_one, df_HVZ_two, df_HVZ_three, df_EP_one, df_EP_two, df_EP_three, df_RI_one, df_RI_two, df_RI_three, X_pred_HVZs, X_pred_EPs, X_pred_RIs, Y_pred_one, Y_pred_two, Y_pred_three = \
    calValues(db, company_list)

Estimated waiting time 4mins, please wait


## Build Model

In [116]:
# Prepare dataset for nine models
y_pred_dict = {}
y_pred_adjusted_dict = {}

# Define a dictionary to store the models
models = {}

# HVZ one year
data = {'HVZ_one': df_HVZ_one, 'HVZ_two': df_HVZ_two, 'HVZ_three': df_HVZ_three,
        'EP_one': df_EP_one, 'EP_two': df_EP_two, 'EP_three': df_EP_three,
        'RI_one': df_RI_one, 'RI_two': df_RI_two, 'RI_three': df_RI_three}

for name, df in data.items(): 
    print(name)
    if 'HVZ' in name:
        X = df[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in name:
        X = df[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    if 'one' in name:
        y = df['E_1']
    elif 'two' in name:
        y = df['E_2']
    elif 'three' in name:
        y = df['E_3']

    X = add_constant(X)
    
    # Define and fit the model
    model = LinearRegression()
    model.fit(X, y)

    # Store the model in the dictionary
    models[name] = model

    # Predict using the model
    y_pred = model.predict(X)
    y_pred_dict[name] = y_pred

    # print('Bias:')
    # print(y - y_pred) 
    # Calculate R-squared
    r_squared = r2_score(y, y_pred)

    # Calculate the adjusted R-squared score
    n = len(X)
    p = len(model.coef_)
    adj_r2 = 1 - (1 - r_squared) * (n - 1) / (n - p - 1)
    print("Adjusted R-squared Score: ", adj_r2)

    # coefficients = pd.DataFrame({"Variable": X.columns, "Coefficient": model.coef_})
    # print(coefficients)

    if 'HVZ' in name:
        y_pred_adjusted = y_pred / df['csho']
        y_pred_adjusted_dict[name] = y_pred_adjusted


HVZ_one
Adjusted R-squared Score:  0.8406273549991103
HVZ_two
Adjusted R-squared Score:  0.7651757641499377
HVZ_three
Adjusted R-squared Score:  0.7206803654926941
EP_one
Adjusted R-squared Score:  0.6517038609160688
EP_two
Adjusted R-squared Score:  0.3762710416336843
EP_three
Adjusted R-squared Score:  0.09523502912320259
RI_one
Adjusted R-squared Score:  0.6586419554490716
RI_two
Adjusted R-squared Score:  0.5278529974434691
RI_three
Adjusted R-squared Score:  0.14284860511722264


In [117]:
# print(Y_pred_one)
# print(Y_pred_two)
# print(Y_pred_three)

In [118]:
# Compare to analyst 
# One, two and three year models
models = {'HVZ_one': models['HVZ_one'], 'EP_one': models['EP_one'], 'RI_one': models['RI_one'],
          'HVZ_two': models['HVZ_two'], 'EP_two': models['EP_two'], 'RI_two': models['RI_two'],
          'HVZ_three': models['HVZ_three'], 'EP_three': models['EP_three'], 'RI_three': models['RI_three']}


for model_name, model in models.items():
    print(model_name)
    if 'HVZ' in model_name:
        X = X_pred_HVZs[['at', 'dvc', 'DD', 'E', 'NegE', 'AC']]
    elif 'EP' in model_name:
        X = X_pred_EPs[['E_deflat', 'NegE', 'NegE_E']]
    elif 'RI' in model_name:
        X = X_pred_RIs[['E_deflat', 'NegE', 'NegE_E', 'B', 'TACC']]
    
    X = add_constant(X)
    
    if 'one' in model_name:
        y = Y_pred_one.copy()
        print(1)
    elif 'two' in model_name:
        y = Y_pred_two.copy()
        print(2)
    elif 'three' in model_name:
        y = Y_pred_three.copy()
        print(3)
    y_pred = model.predict(X)
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
    # y_pred_df = y_pred_df.
    if 'HVZ' in model_name:
        # y_pred_adjusted = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        y_pred_df['y_pred'] = y_pred_df['y_pred'] / X_pred_HVZs['csho']
        # y_pred_adjusted_dict[name] = y_pred_adjusted
        # print(y_pred)
    y_pred_df = pd.concat([X_pred_HVZs['tic'], y_pred_df], axis=1)
    # y_pred_df = pd.concat([y_pred_df, y], axis=1)
    y_pred_df = pd.merge(y_pred_df,y, on= 'tic', how='inner')

    y_pred_df['bias'] = (y_pred_df['E_deflat'] - y_pred_df['y_pred'])

    print(y_pred_df)



HVZ_one
1
       tic    y_pred  E_deflat       bias
0     CGEN  0.255747 -0.354917  -0.610664
1      MBT  1.134208  0.959708    -0.1745
2      KAR  1.593449  0.135698  -1.457751
3      MAN  8.367062  4.018644  -4.348418
4     CENX -0.299086 -1.355823  -1.056737
...    ...       ...       ...        ...
1370    NI  1.776355  1.769706  -0.006649
1371  HURN  2.850485   2.63836  -0.212124
1372    ZD  4.964831  4.332717  -0.632114
1373   CPA  7.896766 -8.301046 -16.197812
1374  APTO  0.239573 -0.621476  -0.861049

[1375 rows x 4 columns]
EP_one
1
       tic     y_pred  E_deflat       bias
0     CGEN -37.694866 -0.354917  37.339949
1      MBT  11.224457  0.959708 -10.264749
2      KAR  10.484509  0.135698 -10.348811
3      MAN -31.058227  4.018644  35.076872
4     CENX -37.961229 -1.355823  36.605406
...    ...        ...       ...        ...
1370    NI   8.091053  1.769706  -6.321347
1371  HURN   4.882909   2.63836  -2.244549
1372    ZD -14.567058  4.332717  18.899775
1373   CPA -31.676137 

In [119]:
y_pred_df['E_deflat'].describe(include='all')

count     1158.000000
unique    1158.000000
top         -0.388964
freq         1.000000
Name: E_deflat, dtype: float64

In [120]:
y_pred_df['E_deflat'].max()

127.03147547840634

In [122]:
from CompanyTextSentimentAnalyzer import TextSentimentAnalyser

# How to use the object
if __name__ == '__main__':
    # 1. Load the dictionary as dataframe
    lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
    # 2. Create a TextSentimentAnalyser object
    text_analyzer = TextSentimentAnalyser(lm_word_list_df)
    # 3. Pass in company ticker and year to calculate sentiment. If company does not have 10k
    # that year it will return null
    sentiment = text_analyzer.get_sentiment('AAPL', 2012)
    print(sentiment)

FileNotFoundError: [Errno 2] No such file or directory: '.\\sec-edgar-filings/AAPL'

In [ ]:
# # df_analyst_one, y_pred_df
# all_prediction = pd.merge(y_pred_df,df_analyst_three, left_on= 'tic', right_on='ticker', how='inner')
# all_prediction['medest'] = all_prediction['medest']

,tic,y_pred,E_deflat,index,ticker,predict_year,fpedats,statpers,actual,medest,gvkey,TICKER
0,CENX,-0.137355,1.673454,1272,CENX,2021.0,2021-12-31,2019-06-20,-0.62,1.27,62516,CENX
1,JKS,-0.502803,1.532952,498,JKS,2021.0,2021-12-31,2019-06-20,1.84,1.13,184182,JKS
2,D,4.874187,1.329341,398,D,2021.0,2021-12-31,2019-06-20,3.86,4.66,4029,D
3,MTD,17.093470,39.88247,2178,MTD,2021.0,2021-12-31,2019-06-20,34.01,28.50,65772,MTD
4,AZPN,3.400586,0.884936,1483,AZPN,2021.0,2021-06-30,2019-06-20,5.20,4.08,30870,AZPN
...,...,...,...,...,...,...,...,...,...,...,...,...
552,DRRX,-0.254264,-1.550713,1311,DRRX,2021.0,2021-12-31,2019-06-20,-1.60,-1.00,140044,DRRX
553,FSLR,3.783309,-3.075519,1571,FSLR,2021.0,2021-12-31,2019-06-20,4.38,4.01,175404,FSLR
554,PEP,4.619980,7.042847,2223,PEP,2021.0,2021-12-31,2019-06-20,6.26,6.44,8479,PEP
555,CDXS,-0.167413,-0.462309,144,CDXS,2021.0,2021-12-31,2019-06-20,-0.33,0.20,179841,CDXS


In [124]:
from CompanyTextSentimentAnalyzer import TextSentimentAnalyser

In [125]:
lm_word_list_df = pd.read_csv('Loughran-McDonald_MasterDictionary_1993-2021.csv')
text_analyzer = TextSentimentAnalyser(lm_word_list_df)

In [127]:
sentiments = []
for company in all_prediction['tic']:
    print(company)
    sentiment = text_analyzer.get_sentiment(company, 2019)
    sentiments = sentiments.append(sentiment, ignore_index=True)
    print(sentiment)


CENX


FileNotFoundError: [Errno 2] No such file or directory: '.\\sec-edgar-filings/CENX'